In [0]:
from datetime import datetime


# Get today's date
today_str = datetime.today().strftime("%Y-%m-%d")

df_test_mgmt = spark.read.parquet(
    f"abfss://ab-test-platform-data@abtestplatform.dfs.core.windows.net/test_management/date={today_str}/"
)

df_test_mgmt = df_test_mgmt.filter(df_test_mgmt.test_status == 4)

df_test_mgmt.show()


+---------+--------------------+-------------------+-----------+-----------+---------------+
|test_code|           test_name|         created_at|finished_at|test_status|     owner_name|
+---------+--------------------+-------------------+-----------+-----------+---------------+
|     T005|Teste de Integraç...|2025-05-11 19:00:00|       NULL|          4|Ricardo Almeida|
+---------+--------------------+-------------------+-----------+-----------+---------------+



In [0]:
df_test_metrics = spark.read.parquet(
    f"abfss://ab-test-platform-data@abtestplatform.dfs.core.windows.net/test_metric_relation/date={today_str}/"
)

df_test_metrics_new = df_test_metrics \
    .withColumn("metric_code_ambiguous", df_test_metrics["metric_code"]) \
        .withColumn("test_code_ambiguous", df_test_metrics["test_code"])

df_test_metrics_new = df_test_metrics_new.drop("metric_code")
df_test_metrics_new = df_test_metrics_new.drop("test_code")

df_test_metrics_new.show()

+----------------+---------------------+-------------------+
|test_metric_code|metric_code_ambiguous|test_code_ambiguous|
+----------------+---------------------+-------------------+
|               1|                 M001|               T004|
|               2|                 M002|               T005|
+----------------+---------------------+-------------------+



In [0]:
df_metric_mgmt = spark.read.parquet(
    f"abfss://ab-test-platform-data@abtestplatform.dfs.core.windows.net/metric_management/date={today_str}/"
)

df_metric_mgmt.show(truncate=False)

+-----------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|metric_code|metric_name     |metric_query                                                                                                                                         |
+-----------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|M001       |Total de cliques|{"query": "SELECT test_code, user_variation, COUNT(user_id) as value FROM table WHERE event_name = 'click' GROUP BY test_code, user_variation"}      |
|M002       |Total de views  |{"query": "SELECT test_code, user_variation, COUNT(user_id) as value FROM table WHERE event_name = 'screen_view' GROUP BY test_code, user_variation"}|
+-----------+----------------+-----------------------------------------------------------------

In [0]:
# Unir df_test_mgmt com df_test_metrics
df_joined_test_metrics = df_test_mgmt.join(
    df_test_metrics_new, 
    df_test_mgmt.test_code == df_test_metrics_new.test_code_ambiguous, 
    "left"
)

df_joined_test_metrics.show()




+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+
|test_code|           test_name|         created_at|finished_at|test_status|     owner_name|test_metric_code|metric_code_ambiguous|test_code_ambiguous|
+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+
|     T005|Teste de Integraç...|2025-05-11 19:00:00|       NULL|          4|Ricardo Almeida|               2|                 M002|               T005|
+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+



In [0]:
# Unir o resultado anterior com df_metric_mgmt
df_final = df_joined_test_metrics.join(
    df_metric_mgmt, 
    df_joined_test_metrics.metric_code_ambiguous == df_metric_mgmt.metric_code, 
    "left"
)

df_final.show()

+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+-----------+--------------+--------------------+
|test_code|           test_name|         created_at|finished_at|test_status|     owner_name|test_metric_code|metric_code_ambiguous|test_code_ambiguous|metric_code|   metric_name|        metric_query|
+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+-----------+--------------+--------------------+
|     T005|Teste de Integraç...|2025-05-11 19:00:00|       NULL|          4|Ricardo Almeida|               2|                 M002|               T005|       M002|Total de views|{"query": "SELECT...|
+---------+--------------------+-------------------+-----------+-----------+---------------+----------------+---------------------+-------------------+-----------+--------------+--------------------+


In [0]:
# Selecionar as colunas desejadas
df_result = df_final.select(
    "test_code", 
    "test_metric_code", 
    "metric_code", 
    "metric_name", 
    "metric_query"
)

# Mostrar o resultado
df_result.show()

+---------+----------------+-----------+--------------+--------------------+
|test_code|test_metric_code|metric_code|   metric_name|        metric_query|
+---------+----------------+-----------+--------------+--------------------+
|     T005|               2|       M002|Total de views|{"query": "SELECT...|
+---------+----------------+-----------+--------------+--------------------+

+---------+----------------+-----------+--------------+--------------------+
|test_code|test_metric_code|metric_code|   metric_name|        metric_query|
+---------+----------------+-----------+--------------+--------------------+
|     T002|               2|       M002|Total de views|{"query": "SELECT...|
+---------+----------------+-----------+--------------+--------------------+



In [0]:
test_users = spark.table("db_experiments.test_users")

test_users.show()



+-------+---------+--------------+--------------+
|user_id|test_code|user_variation|ingestion_date|
+-------+---------+--------------+--------------+
|   U092|     T002|             A|    2025-05-11|
|   U064|     T001|             A|    2025-05-11|
|   U083|     T002|             B|    2025-05-11|
|   U081|     T002|             A|    2025-05-11|
|   U070|     T001|             A|    2025-05-11|
|   U075|     T001|             A|    2025-05-11|
|   U030|     T001|             B|    2025-05-11|
|   U036|     T002|             B|    2025-05-11|
|   U026|     T002|             B|    2025-05-11|
|   U025|     T002|             B|    2025-05-11|
|   U001|     T001|             A|    2025-05-11|
|   U057|     T001|             A|    2025-05-11|
|   U039|     T002|             B|    2025-05-11|
|   U048|     T002|             A|    2025-05-11|
|   U058|     T002|             B|    2025-05-11|
|   U038|     T002|             B|    2025-05-11|
|   U076|     T001|             B|    2025-05-11|


In [0]:
business_events = spark.table("db_experiments.business_events")

business_events.show()

+--------------------+-------+-------------------+-----------+----------+------------+----------+
|            event_id|user_id|         event_time| event_name|event_path|event_device|event_date|
+--------------------+-------+-------------------+-----------+----------+------------+----------+
|464d04b6-1e59-4d3...|   U010|2025-05-09 04:07:10|     scroll| /purchase|      mobile|2025-05-09|
|23a6f7b6-dbe1-4a8...|   U002|2025-05-11 10:39:10|      click|     /home|      mobile|2025-05-11|
|a29693a3-52e9-43f...|   U019|2025-05-10 02:09:10|screen_view|    /login|     desktop|2025-05-10|
|64fd4e85-cab4-4ae...|   U033|2025-05-10 17:41:10|screen_view|     /home|     desktop|2025-05-10|
|fdf3154a-02b6-42e...|   U099|2025-05-08 23:54:10|      click|     /cart|     desktop|2025-05-08|
|e5feaa2b-3317-4ba...|   U015|2025-05-10 10:45:10|     scroll| /purchase|     desktop|2025-05-10|
|09f276e8-fa17-48e...|   U077|2025-05-08 21:28:10|     scroll|     /home|     desktop|2025-05-08|
|1ec9cedb-65ee-40d..

In [0]:
import json
from pyspark.sql.functions import lit

for row in df_result.collect():
    test_code = row['test_code']
    metric_query_json = row['metric_query']
    metric_query = json.loads(metric_query_json)['query']  # extrai a string da query

    print(f"Executando métrica para Teste {test_code}: {row['metric_name']}")

    # 1. Filtrar usuários do teste
    df_users_test = test_users.filter(test_users.test_code == test_code)

    # 2. Fazer left join com business_events
    df_joined = df_users_test.join(
        business_events,
        on="user_id",
        how="left"
    )

    # 3. Registrar como view temporária para rodar SQL
    df_joined.createOrReplaceTempView("table")

    # 4. Executar a métrica
    result_df = spark.sql(metric_query)
    result_df.show()

    # Suponha que result_df é o resultado da métrica, com colunas:
    # test_code | user_variation | value

    metric_code = row['metric_code']
    metric_name = row['metric_name']
    metric_date = datetime.now().strftime('%Y-%m-%d')

    # Adicionar as colunas restantes
    df_final_metric = result_df.withColumnRenamed("user_variation", "variation") \
        .withColumnRenamed("value", "metric_value") \
        .withColumn("metric_code", lit(metric_code)) \
        .withColumn("metric_name", lit(metric_name)) \
        .withColumn("metric_date", lit(metric_date)) \
        .select(
            "test_code",
            "metric_code",
            "metric_name",
            "variation",
            "metric_value",
            "metric_date"
        )

    # Exibir resultado final
    df_final_metric.show()

    df_final_metric.write.mode("append").insertInto("db_experiments.metric")



Executando métrica para Teste T002: Total de views
+---------+--------------+-----+
|test_code|user_variation|value|
+---------+--------------+-----+
|     T002|             B| 1906|
|     T002|             A| 1700|
+---------+--------------+-----+

+---------+-----------+--------------+---------+------------+-----------+
|test_code|metric_code|   metric_name|variation|metric_value|metric_date|
+---------+-----------+--------------+---------+------------+-----------+
|     T002|       M002|Total de views|        B|        1906| 2025-05-11|
|     T002|       M002|Total de views|        A|        1700| 2025-05-11|
+---------+-----------+--------------+---------+------------+-----------+

